In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pycountry
import category_encoders as ce
import math
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
%matplotlib inline

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_rows', 200)

In [ ]:
RANDOM_SEED = 28
DATA_DIR = '/kaggle/input/sf-booking/'
WORD_POPULATION_DIR = '/kaggle/input/world-population-by-city-without-nans/'

# 1. Загрузка данных

In [ ]:
df_train = pd.read_csv(DATA_DIR + 'hotels_train.csv')
df_test = pd.read_csv(DATA_DIR + 'hotels_test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'submission.csv')

In [ ]:
df_train.head(3)

In [ ]:
sample_submission.head()

In [ ]:
# Объединяем train и test данные в одну таблицу
df_train['is_train'] = 1
df_test['is_train'] = 0
df_test['reviewer_score'] = 0

data = df_test.append(df_train, sort=False).reset_index(drop=True)

In [ ]:
data.info()

Видно, что данные почти не имеют пропусков кроме признаков: lat, lng. Пока не очень понятно как их заполнять, для этого попробуем изучить признак hotel_address, выделить оттуда адрес и на его основе попробовать правильно заполнить lat, lng.

# 2. Исследование и проектирование признаков

## 2.1. hotel_address

In [ ]:
display(pd.Series(data['hotel_address'].unique()))

Видно, что из строки адреса можно извлечь страну, город и индекс района.

In [ ]:
# Функция для нахождения названия страны в тексте
def find_country(text):
    for country in pycountry.countries:
        if country.name in text:
            return country.name

# Проходим по каждой строке в столбце 'hotel_address', находим название страны и добавляем в отдельный признак
data['hotel_country'] = data['hotel_address'].apply(find_country)
display(data['hotel_country'].value_counts())

Видим, что всего в данных оказывается 6 стран, причем большинство отелей из United Kingdom.

In [ ]:
# Для извлечения городов из текста я воспользовался данными из проекта world-population-by-city-without-nans
world_cities = pd.read_csv(WORD_POPULATION_DIR + 'worldcitiespop.csv')
display(world_cities.head())
display(world_cities.info())
# Создаем список городов без дубликатов
cities = set(world_cities['City'].tolist())

# Функция для нахождения названия города в тексте
def find_city(address):
    address = address.lower().split(' ')[::-1]
    address = address[1:]
    for word in address:
        if word == 'united':
            continue
        elif word in cities:
            return word

# Проходим по каждой строке в столбце 'hotel_address', находим название города и добавляем в отдельный признак
data['hotel_city'] = data['hotel_address'].apply(find_city)
display(data['hotel_city'].value_counts())

Так же в данных оказывается 6 городов по одному на каждую страну. В данных из проекта world-population-by-city-without-nans есть признаки lat, lng для каждого города, от них и рассчитаем расстояния до отелей в которых пропущены эти данные.

In [ ]:
# Словарь для сопоставления городов и стран
city_country_map = {
    'london': 'gb',
    'barcelona': 'es',
    'paris': 'fr',
    'amsterdam': 'nl',
    'vienna': 'at',
    'milan': 'it'
}
# Координаты присутствующих в данных городов
city_coordinates = pd.DataFrame({
    'hotel_city': list(city_country_map.keys()),
    'cnt_lat': [None] * len(city_country_map), 
    'cnt_lng': [None] * len(city_country_map)
})

# Проход по каждому элементу словаря для заполнения координат
for i, (city, country) in enumerate(city_country_map.items()):
    # Поиск города и страны
    city_data = world_cities[(world_cities['City'].str.lower() == city.lower()) & 
                             (world_cities['Country'].str.lower() == country.lower())]
    
    # Если город и страна найдены, заполняем координаты
    if not city_data.empty:
        city_coordinates.at[i, 'cnt_lat'] = city_data.iloc[0]['Latitude']
        city_coordinates.at[i, 'cnt_lng'] = city_data.iloc[0]['Longitude']

# Полученную таблицу присоединим к основной таблице данных
data = data.merge(city_coordinates, on='hotel_city', how='left')
display(data.head(3))

In [ ]:
# Кодируем информацию о стране
encoder = ce.OneHotEncoder(cols=['hotel_country']) 
type_bin = encoder.fit_transform(data['hotel_country'])
data = pd.concat([data, type_bin], axis=1)

# Кодируем информацию о городе
encoder = ce.OneHotEncoder(cols=['hotel_city']) 
type_bin = encoder.fit_transform(data['hotel_city'])
data = pd.concat([data, type_bin], axis=1)

Построим график средних оценок по городам

In [ ]:
rating_country = data.groupby(by='hotel_city')[['average_score']].mean()

fig, ax = plt.subplots(figsize=[14, 8])
ax.bar(rating_country.index, rating_country['average_score'], color=['r', 'g', 'b', 'y', 'm', 'orange'])
ax.set_ylabel('average_score')
ax.set_title('Средняя оценка по городам')
plt.ylim(8, 8.8)
plt.xticks(rotation = 45)
plt.grid()
plt.show();

Из графика видно, что средняя оценка меняется в зависимости от города в котром находится отель.

### 2.1.1. Заполнение пропущенных значений lat, lng

In [ ]:
# Функция визуализации распределения пропусков в данных по столбцам
def missing_values_heatmap(data):
    cols_null_persent = data.isnull().mean() * 100
    cols_with_null = cols_null_persent[cols_null_persent>0].sort_values(ascending=False)
    colors = ['blue', 'yellow'] 
    fig = plt.figure(figsize=(10, 4))
    cols = cols_with_null.index
    ax = sns.heatmap(data[cols].isnull(), cmap=sns.color_palette(colors))
    return ax

missing_values_heatmap(data)

In [ ]:
# Находим пропуски
missing_data = data[(data['lat'].isna() == True) | (data['lng'].isna() == True)]
# Заполняем столбцы lat и lng значениями координат столиц
missing_data['lat'] = missing_data['cnt_lat']
missing_data['lng'] = missing_data['cnt_lng']
data[(data['lat'].isna() == True) | (data['lng'].isna() == True)] = missing_data

Посчитаем расстояние от центра города до отелей и определим категорию расстояния.

In [ ]:
# Функция присвоения категории в зависимости от расстояния
def distance_category(arg):
    if arg < 1:
        return 'less 1'
    
    elif arg < 3:
        return 'less 3'
    
    elif arg < 5:
        return 'less 5'
    
    elif arg >= 5:
        return 'more 5'
    
data['distance'] = float(0)
R = 6373.0
rows = data.index
for row in rows:
    lat1 = math.radians(data['lat'][row])
    lat2 = math.radians(data['cnt_lat'][row])
    lng1 = math.radians(data['lng'][row])
    lng2 = math.radians(data['cnt_lng'][row])
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlng / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    data['distance'][row] = distance

# Для отелей расстояние которых от центра города равно 0 присваиваем среднее расстояние
data[data['distance'] == 0] = data[data['distance'] == 0].assign(distance=3.25)
# Добавим категориальный признак расстояния от центра города
data['distance_cat'] = data['distance'].apply(distance_category)
display(data.head(3))

In [ ]:
# Кодируем информацию о расстоянии
encoder = ce.OneHotEncoder(cols=['distance_cat']) 
type_bin = encoder.fit_transform(data['distance_cat'])
data = pd.concat([data, type_bin], axis=1)

Построим график средних оценок в зависимости от расстояния

In [ ]:
rating_country = data.groupby(by='distance_cat')[['average_score']].mean()

fig, ax = plt.subplots(figsize=[14, 8])
ax.bar(rating_country.index, rating_country['average_score'], color=['r', 'g', 'b', 'y'])
ax.set_ylabel('average_score')
ax.set_title('Средняя оценка по расстоянию')
plt.ylim(8, 8.8)
plt.xticks(rotation = 45)
plt.grid()
plt.show();

Получается чем меньше расстояние от центра тем выше оценка, что является логичным.

## 2.2. reviewer_nationality

In [ ]:
display(dict(data['reviewer_nationality'].value_counts()))

Видим огромный список стран, многие из которых похоже нужно объединять в одну категорию плюс из некоторых стран буквально единицы клиентов.

In [ ]:
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x.strip())
# Добавим признак резидентства туриста по отношению к стране, в которой находится отель
data['reviewer_resident'] = data.apply(lambda row: 1 if row['reviewer_nationality'] == row['hotel_country'] else 0, axis=1)

# Кодируем информацию о стране рецензента
data['reviewer_nationality'] = data['reviewer_nationality'].astype('category')
data['reviewer_nationality_cat'] = data['reviewer_nationality'].cat.codes

Построим график соотношения средних показателей оценок туристов в разрезе новых признаков

In [ ]:
# Строим данные для графика
mask = data['is_train'] == 1
pivot_resident = data[mask].groupby('reviewer_resident')['reviewer_score'].mean()
pivot = pd.DataFrame({'путешествует по своей стране': pivot_resident}).T

# Выводим график
fig2, ax2 = plt.subplots(figsize=(15, 5))
pivot_barplot = pivot.plot(
    ax=ax2, 
    kind='bar', 
    color=['mediumslateblue', 'seagreen'], 
    rot=0)
ax2.set_title('Соотношение средних показателей оценок туристов в разрезе новых признаков', size=16)
ax2.legend(['нет', 'да'], loc='upper right', fontsize=14)

for p in pivot_barplot.patches:
    pivot_barplot.annotate('{:.2f}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()),
    ha='center', va='bottom', fontsize=18)

plt.ylim(7, 9.5);

Как видно из графика пропорция путешествующих по своей/чужой стране примерно равна

## 2.3. review_date

In [ ]:
# Преобразуем 'review_date' в datetime формат
data['review_date'] = pd.to_datetime(data['review_date'])

# Отдельно выделяем: месяц, год, день
data['month'] = pd.to_datetime(data['review_date']).dt.month
data['year'] = pd.to_datetime(data['review_date']).dt.year
data['day'] = pd.to_datetime(data['review_date']).dt.day

# Добавим признаки будних и выходных
data['weekday'] = pd.to_datetime(data['review_date']).dt.weekday
data['weekend'] = data['weekday'].apply(lambda day: 1 if day==6 or day==5 else 0)

Построим график средней оценки по отзывам в зависимости от времени года

In [ ]:
# Строим данные для графика
mask = data['is_train'] == 1
pivot_month = data[mask][['month', 'reviewer_score']].groupby('month').mean()

# Выводим график
fig3, ax3 = plt.subplots(figsize=(15, 5))
plt.suptitle('Средняя оценка по отзывам в зависимости от времени года', size=16)
bar_month = sns.barplot(
    x=pivot_month.index, 
    y=pivot_month['reviewer_score'])
ax3.set_xticklabels([
    'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 
    'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'])

for p in bar_month.patches:
    bar_month.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x()+0.4, p.get_height()),
    ha='center', va='bottom', fontsize=14)

plt.ylim(8, 8.7);

Получаем результат, что лучшее время для туризма в европе это зимние месяцы плюс март и апрель. Худшие - лето и осень.
Теперь построим график средней оценки по отзывам в зависимости от дня недели.

In [ ]:
# Строим данные для графика
mask = data['is_train'] == 1
pivot_month = data[mask][['weekday', 'reviewer_score']].groupby('weekday').mean()

# Выводим график
fig4, ax4 = plt.subplots(figsize=(15, 5))
plt.suptitle('Средняя оценка по отзывам в зависимости дня недели', size=16)
bar_month = sns.barplot(
    x=pivot_month.index, 
    y=pivot_month['reviewer_score'])
ax4.set_xticklabels([
    'Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 
    'Воскресение'])

for p in bar_month.patches:
    bar_month.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x()+0.4, p.get_height()),
    ha='center', va='bottom', fontsize=14)

plt.ylim(8, 8.7);

Чуть лучшие результаты получились для вторника, среды и воскресения.

## 2.4. positive_review и negative_review

In [ ]:
# Приводим отзывы к более однозначному варианту
data['negative_review'] = data['negative_review'].apply(lambda x: x.replace('No Negative', 'Positive'))
data['negative_review'] = data['negative_review'].apply(lambda x: x.replace('N A', 'Positive'))
data['negative_review'] = data['negative_review'].apply(
    lambda x: x.replace('All good', 'Positive'))
data['negative_review'] = data['negative_review'].apply(
    lambda x: x.replace('No complaints', 'Positive'))
data['negative_review'] = data['negative_review'].apply(
    lambda x: x.replace('Nothing to dislike', 'Positive'))
data['positive_review'] = data['positive_review'].apply(
    lambda x: x.replace('No Positive', 'Negative'))

# Для анализа положительных и отрицательных отзывов создаем два анализатора
neg_analyzer = SentimentIntensityAnalyzer()
pos_analyzer = SentimentIntensityAnalyzer()

data['neg_score'] = data['negative_review'].apply(lambda x: neg_analyzer.polarity_scores(x))
data['pos_score'] = data['positive_review'].apply(lambda x: pos_analyzer.polarity_scores(x))
display(data[['neg_score', 'pos_score']])

Для примера выведем облаго тегов для категории - negative_review

In [ ]:
# Визуализация отзывов в категории negative_review
wordcloud = WordCloud(
    background_color = 'white', 
    colormap = 'BrBG', 
    max_font_size = 40, 
    max_words=100, 
    scale = 3, 
    random_state = 42
).generate(str(data['negative_review']))

plt.figure(1, figsize = (20, 20))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

In [ ]:
# На основе полученного результата анализа создаем новые признаки
data['n_review_sentiments_neg'] = data['neg_score'].apply(lambda x: x['neg'])
data['n_review_sentiments_neu'] = data['neg_score'].apply(lambda x: x['neu'])
data['n_review_sentiments_pos'] = data['neg_score'].apply(lambda x: x['pos'])
data['n_review_sentiments_compound'] = data['neg_score'].apply(lambda x: x['compound'])

data['p_review_sentiments_neg'] = data['pos_score'].apply(lambda x: x['neg'])
data['p_review_sentiments_neu'] = data['pos_score'].apply(lambda x: x['neu'])
data['p_review_sentiments_pos'] = data['pos_score'].apply(lambda x: x['pos'])
data['p_review_sentiments_compound'] = data['pos_score'].apply(lambda x: x['compound'])

## 2.5. tags

In [ ]:
# Извлекаем и очищаем теги
def tag_clean(arg):
    arg = arg[2:-2].split('\', \'')
    return arg

data['new_tags'] = data['tags'].apply(tag_clean)
df = data.explode('new_tags')
# Выбираем 30 наиболее часто используемых
tags_counts = df['new_tags'].value_counts()[:30]
df_tags_counts = pd.DataFrame(tags_counts).reset_index()
df_tags_counts.columns = ['unique_tags', 'counts_unique_tags']
top_tags = set(df_tags_counts['unique_tags'])

In [ ]:
# Визуализация топ 30 тегов
wordcloud = WordCloud(
    background_color = 'white', 
    colormap = 'BrBG', 
    max_font_size = 40, 
    max_words=100, 
    scale = 3, 
    random_state = 42
).generate(str(top_tags))

plt.figure(1, figsize = (20, 20))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

In [ ]:
# Создаем новый признак - если тег присутствует в топ 30 выставляем значение в 1 иначе 0
for tag in top_tags:
    tag_name = str(tag)
    data[tag_name] =  data['tags'].apply(lambda x: 1 if tag_name in x else 0)

# 3. Корреляционный анализ

Проверим признаки на мультиколлиниарность и удалим признаки с самой высокой корреляцией

In [ ]:
corr = data.corr()
corr_unstack = corr.abs().unstack().reset_index()
corr_unstack = corr_unstack.sort_values(by=[0], ascending = False)
mask = corr_unstack['level_0'] != corr_unstack['level_1']
corr_unstack = corr_unstack[mask]
corr_unstack['pr1'] = corr_unstack['level_0'] + corr_unstack['level_1']
corr_unstack['pr1'] = corr_unstack['pr1'].apply(lambda x: ''.join(sorted(list(x))))
corr_unstack = corr_unstack.drop_duplicates(subset=['pr1'])
corr_unstack = corr_unstack.drop(['pr1'], axis=1)
display(corr_unstack.head(25))

In [ ]:
# Удаляем лишние признаки
columns_del = ['hotel_address',
               'hotel_name',
               'review_date',
               'days_since_review',
               'reviewer_nationality', 
               'negative_review', 
               'day',
               'positive_review', 
               'tags', 
               'lat', 
               'lng', 
               'hotel_country', 
               'hotel_city', 
               'cnt_lat', 
               'cnt_lng', 
               'distance_cat', 
               'neg_score', 
               'pos_score', 
               'new_tags', 
               'distance', 
               ' Twin Room '
              ]
new_data = data.drop(columns=columns_del, axis=1)

In [ ]:
# Визуализируем тепловую карту корреляций
plt.rcParams['figure.figsize'] = (30,20)
sns.heatmap(corr, annot = True,  fmt='.1f');

# 4. Обучение

In [ ]:
test_data = new_data[new_data['is_train'] == 0]
train_data = new_data[new_data['is_train'] == 1]
train_data = train_data.drop(['is_train'], axis=1)
test_data = test_data.drop(['is_train'], axis=1)

y = train_data.reviewer_score.values
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Используем функциею train_test_split для разделения тестовых данных выделим 20% данных на валидацию
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

model = RandomForestRegressor(
    n_estimators=100, 
    verbose=1, 
    n_jobs=-1, 
    random_state=RANDOM_SEED
)

# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [ ]:
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))